In [32]:
pip install nltk


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
#import important libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import string
import re, os, io
from numpy import array, argmax, random, take
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', None)


In [34]:
import tensorflow as tf
tf.__version__

'2.14.0'

In [35]:
import tensorflow_addons as tfa

In [36]:
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers

In [37]:
filepath_en_1 = "data1/english.txt"
filepath_te_1 = "data1/telugu.txt"
filepath_en_2 = "data2/english.txt"
filepath_te_2 = "data2/telugu.txt"

In [38]:
tel_text_1  = io.open(filepath_te_1,encoding='UTF-8').read().strip().split('\n')
eng_text_1  = io.open(filepath_en_1).read().strip().split('\n')
print("Number of Examples in Telugu Language Data1:",len(tel_text_1))
print("Number of Examples in English Language Data1:",len(eng_text_1))

tel_text_2  = io.open(filepath_te_2,encoding='UTF-16').read().strip().split('\n')
eng_text_2  = io.open(filepath_en_2).read().strip().split('\n')
print("Number of Examples in Telugu Language Data2:",len(tel_text_2))
print("Number of Examples in English Language Data2:",len(eng_text_2))

Number of Examples in Telugu Language Data1: 5000
Number of Examples in English Language Data1: 5000
Number of Examples in Telugu Language Data2: 4499
Number of Examples in English Language Data2: 4499


In [39]:
sample_tel_1 = tel_text_1[0:5]
sample_eng_1 = eng_text_1[0:5]
print(sample_tel_1)
print()
print(sample_eng_1)

sample_tel_2 = tel_text_2[0:5]
sample_eng_2 = eng_text_2[0:5]
print(sample_tel_2)
print()
print(sample_eng_2)

["ఈ సందర్భంగా టీమిండియా మాజీ కెప్టెన్\u200c 'ది వాల్' రాహుల్\u200c ద్రావిడ్\u200c తన ఫేవరెట్\u200c క్రికెటర్\u200c అని దీపికా తెలిపారు.", 'డ్రైనెస్ ను తొలగించాలి.', 'శ్రీదేవి ఆకస్మిక మరణం దేశవ్యాప్తంగా ప్రతీ ఒక్కరిని శోక సంద్రంలో ముంచెత్తింది.', 'అలాంటి యత్నమే ఇది.', '"5 మెగా పిక్సల్ ఫ్రట్ ఫేసింగ్ కెమెరా,"']

['"speaking about her all-time favourite cricketer, deepika said its former india skipper rahul dravid."', 'debris has to be removed.', "sridevi's sudden demise sent shock waves throughout the country.", 'this is one such attempt.', '5-megapixel front camera']
['"రామాయణం భారతీయ వాఙ్మయంలో ఆదికావ్యంగాను, దానిని సంస్కృతంలో రచించిన వాల్మీకి మహాముని ఆదికవిగాను సుప్రసిద్ధం."', '"సాహిత్య చరిత్ర ప్రకారం రామాయణ కావ్యం వేద కాలం తర్వాత, అనగా సుమారు సా.శ."', 'రామాయణం కావ్యంలోని కథ త్రేతాయుగం కాలంలో జరిగినట్లు వాల్మీకి పేర్కొన్నాడు.', '"భారతదేశం లోని అన్ని భాషల యందు, అన్ని ప్రాంతాలనందు ఈ కావ్యం ఎంతో ఆదరణీయం, పూజనీయం."', '"ఇండోనేషియా, థాయిలాండ్, కంబోడియా, మలేషియా, వియత్నాం, లావోస్ దేశాలలో కూడా 

In [40]:
import unicodedata

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!='BN')

# Remove punctuations, reduce words to lower case,and adding '<start>' and '<end>' tokens to the sentences

def preprocess_eng(w):
    #w= unicode_to_ascii(w)
    w= w.lower().strip()
    w= re.sub(r'([?.!,¿_])',r' \1 ',w)
    w= re.sub(r'[" "]+', " ",w) # keep white space
    w= re.sub(r"[^a-zA-Z?.!,¿_]+"," ",w)
    w= w.strip()
    w= '<start> '+ w +' <end>'
    return w

def preprocess_tel(w):
    w= unicode_to_ascii(w)
    w= re.sub(r'([\?.!,¿_])',r' \1 ',w)
    w= re.sub(r'[" "]+', " ",w) # keep white space
    w= w.strip()
    w= '<start> '+ w +' <end>'
    w= w.replace("u'", "'")
    return w

line=[preprocess_eng(i) for i in sample_eng_1]
tuple(line)



('<start> speaking about her all time favourite cricketer , deepika said its former india skipper rahul dravid . <end>',
 '<start> debris has to be removed . <end>',
 '<start> sridevi s sudden demise sent shock waves throughout the country . <end>',
 '<start> this is one such attempt . <end>',
 '<start> megapixel front camera <end>')

In [41]:
def create_eng_dataset(filepath):
    #limit num_examples for faster training - len(lines) is full data
    lines= io.open(filepath,encoding='UTF-8').read().strip().split('\n')
    lines=[preprocess_eng(i) for i in lines]
    return tuple(lines)


def create_tel_dataset(filepath):
    #limit num_examples for faster training - len(lines) is full data
    if filepath==filepath_te_1:
        lines= io.open(filepath,encoding='UTF-8').read().strip().split('\n')
    else:
        lines= io.open(filepath,encoding='UTF-16').read().strip().split('\n')
    lines=[preprocess_tel(i) for i in lines]
    return tuple(lines)

In [42]:
# tokenize function 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional

def tokenize(lang):
    lang_tokenizer = Tokenizer(filters='',oov_token='<OOV>')
    lang_tokenizer.fit_on_texts(lang)
    lang_data = lang_tokenizer.texts_to_sequences(lang)
    lang_data = pad_sequences(lang_data,padding="post")
    return lang_data, lang_tokenizer

def load_dataset(filepath1,filepath2):
    # create cleaned input and target sentences
    input_lang = create_eng_dataset(filepath1)
    targ_lang  = create_tel_dataset(filepath2)
    input_lang, inp_lang_tokenizer  = tokenize(input_lang)
    target_lang, targ_lang_tokenizer= tokenize(targ_lang)
    return input_lang,target_lang,inp_lang_tokenizer,targ_lang_tokenizer

In [43]:

input_lang_1,target_lang_1,inp_lang_tokenizer_1,targ_lang_tokenizer_1 = load_dataset(filepath_en_1,filepath_te_1)

input_lang_2,target_lang_2,inp_lang_tokenizer_2,targ_lang_tokenizer_2 = load_dataset(filepath_en_2,filepath_te_2)

In [44]:
input_lang_1.shape,target_lang_1.shape
print(input_lang_2.shape,target_lang_2.shape)
input_lang_1[0].shape
print(input_lang_2[0].shape)

(4499, 92) (4499, 75)
(92,)


In [45]:
from sklearn.model_selection import train_test_split

input_train_1, input_val_1, target_train_1, target_val_1 = train_test_split(input_lang_1, target_lang_1, test_size=0.2)
input_train_2, input_val_2, target_train_2, target_val_2 = train_test_split(input_lang_2, target_lang_2, test_size=0.2)

In [46]:
input_train_1.shape,target_train_1.shape

((4000, 92), (4000, 95))

In [47]:

input_train_2.shape,target_train_2.shape


((3599, 92), (3599, 75))

In [48]:
buffer_size = 5000
batch_size  = 64

train_dataset_1 = tf.data.Dataset.from_tensor_slices((input_train_1,target_train_1))
train_dataset_1 = train_dataset_1.shuffle(buffer_size).batch(batch_size,drop_remainder =True)

train_dataset_2 = tf.data.Dataset.from_tensor_slices((input_train_2,target_train_2))
train_dataset_2 = train_dataset_2.shuffle(buffer_size).batch(batch_size,drop_remainder =True)

train_dataset_1
train_dataset_2

<_BatchDataset element_spec=(TensorSpec(shape=(64, 92), dtype=tf.int32, name=None), TensorSpec(shape=(64, 75), dtype=tf.int32, name=None))>

In [49]:
val_dataset_1 = tf.data.Dataset.from_tensor_slices((input_val_1,target_val_1))
val_dataset_1 = val_dataset_1.batch(batch_size,drop_remainder =True)

val_dataset_2 = tf.data.Dataset.from_tensor_slices((input_val_2,target_val_2))
val_dataset_2 = val_dataset_2.batch(batch_size,drop_remainder =True)

In [50]:
ex_input_batch_1,ex_tar_batch_1 = next(iter(train_dataset_1))
ex_input_batch_1.shape,ex_tar_batch_1.shape

maxlen_input_1 = ex_input_batch_1.shape[1]
maxlen_output_1   = ex_tar_batch_1.shape[1]

vocab_inp_size_1 = len(inp_lang_tokenizer_1.word_index)+1
vocab_tar_size_1 = len(targ_lang_tokenizer_1.word_index)+1

ex_input_batch_2,ex_tar_batch_2 = next(iter(train_dataset_2))
ex_input_batch_2.shape,ex_tar_batch_2.shape

maxlen_input_2 = ex_input_batch_2.shape[1]
maxlen_output_2   = ex_tar_batch_2.shape[1]

vocab_inp_size_2 = len(inp_lang_tokenizer_2.word_index)+1
vocab_tar_size_2 = len(targ_lang_tokenizer_2.word_index)+1

embedding_dim = 256
units = 512
steps_per_epoch_1 = 5000//batch_size
steps_per_epoch_2= 4270//batch_size



print("Dataset2: max_length_english, max_length_telugu, vocab_size_english, vocab_size_telugu")
print(maxlen_input_2, maxlen_output_2, vocab_inp_size_2, vocab_tar_size_2)

Dataset2: max_length_english, max_length_telugu, vocab_size_english, vocab_size_telugu
92 75 5949 11797


In [51]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm_layer = tf.keras.layers.LSTM(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, h, c = self.lstm_layer(x, initial_state=hidden)
        return output, h, c
    
    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_size, self.enc_units)), tf.zeros((self.batch_size, self.enc_units))]  # all zeros


In [52]:
## Test Encoder Stack


encoder_2 = Encoder(vocab_inp_size_2, embedding_dim, units,batch_size)


# sample input
sample_hidden_2 = encoder_2.initialize_hidden_state()
sample_output_2, sample_h_2, sample_c_2 = encoder_2(ex_input_batch_2, sample_hidden_2)
print ('Dataset 2')
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output_2.shape))
print ('Encoder h vecotr shape: (batch size, units) {}'.format(sample_h_2.shape))
print ('Encoder c vector shape: (batch size, units) {}'.format(sample_c_2.shape))

Dataset 2
Encoder output shape: (batch size, sequence length, units) (64, 92, 512)
Encoder h vecotr shape: (batch size, units) (64, 512)
Encoder c vector shape: (batch size, units) (64, 512)


In [53]:
import gc
gc.collect()

628

In [54]:
np.random.seed(42)
tf.random.set_seed(42)

encoder_embedding_dim = 256
decoder_embedding_dim = 256
units=512
batch_size =64
maxlen_output_1 = target_train_1.shape[1]
maxlen_output_2 = target_train_2.shape[1]

In [55]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.units = units
        
        # Embedding Layer
        self.decoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        #Final Dense layer on which softmax will be applied
        self.output_layer = tf.keras.layers.Dense(vocab_size)
        
        # Define the fundamental cell for decoder recurrent structure
        self.decoder_inner_cell = tf.keras.layers.LSTMCell(self.units)
        
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        
        ## Attention Wrapper ##
        self.attention_mechanism = tfa.seq2seq.attention_wrapper.LuongAttention(units,memory=None,
                                                                                memory_sequence_length=self.batch_size*[maxlen_input_2])
        self.attention_decoder_cell = tfa.seq2seq.attention_wrapper.AttentionWrapper(cell=self.decoder_inner_cell, 
                                                                                     attention_mechanism=self.attention_mechanism,
                                                                                     attention_layer_size=self.units)
        
        ## Basic Decoder ##
        self.decoder = tfa.seq2seq.BasicDecoder(cell=self.attention_decoder_cell,sampler = self.sampler,
                                               output_layer = self.output_layer)
        
         
        self.inference_decoder = tfa.seq2seq.BasicDecoder(cell=self.attention_decoder_cell,
                                                          sampler = tfa.seq2seq.sampler.GreedyEmbeddingSampler(
                                                              embedding_fn = self.decoder_embedding),
                                                          output_layer=self.output_layer,maximum_iterations = 154)
        
    def build_initial_state(self, batch_size, encoder_state, Dtype):
        decoder_initial_state = self.attention_decoder_cell.get_initial_state(batch_size=batch_size, dtype=Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
        return decoder_initial_state
        
    def call(self,decoder_input,initial_state):
        decoder_embeddings = self.decoder_embedding(decoder_input)
        outputs, _, _ = self.decoder(decoder_embeddings,initial_state=initial_state,
                                     sequence_length=self.batch_size*[maxlen_output_2-1])
        return outputs

In [56]:
# Test decoder stack

decoder_2 = Decoder(vocab_tar_size_2, embedding_dim,512, batch_size)
sample_x_2 = tf.random.uniform((batch_size, maxlen_output_2))
decoder_2.attention_mechanism.setup_memory(sample_output_2) # from encoder
initial_state_1 = decoder_2.build_initial_state(batch_size, [sample_h_2, sample_c_2], tf.float32)

sample_decoder_outputs_2 = decoder_2(sample_x_2, initial_state_1)

print("Decoder1 Outputs Shape: ", sample_decoder_outputs_2.rnn_output.shape)


Decoder1 Outputs Shape:  (64, 74, 11797)


In [57]:
gc.collect()

0

In [58]:
#Optimizer

optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
    # real shape = (batch_size, maxlen_output)
    # pred shape = (batch_size, maxlen_output, vocab_tar_size )
    cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    loss = cross_entropy(y_true=real, y_pred=pred)
    mask = tf.logical_not(tf.math.equal(real,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)  
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss

In [59]:
checkpoint_dir_2 = './training_checkpoints_1'
checkpoint_prefix_2 = os.path.join(checkpoint_dir_2, "ckpt")
checkpoint_2 = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder_2,
                                 decoder=decoder_2)

In [ ]:
@tf.function
def train_step(encoder, decoder,inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)
        
        dec_input = targ[ : , :-1 ] # Ignore <end> token
        real = targ[ : , 1: ]         # ignore <start> token
        
        # Set the AttentionMechanism object with encoder_outputs
        decoder.attention_mechanism.setup_memory(enc_output)
        # Create AttentionWrapperState as initial_state for decoder
        decoder_initial_state = decoder.build_initial_state(batch_size, [enc_h, enc_c], tf.float32)
        pred = decoder(dec_input, decoder_initial_state)
        logits = pred.rnn_output
        loss = loss_function(real, logits)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss



class ReplayBuffer:
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.buffer = []

    def add(self, experience):
        if len(self.buffer) >= self.buffer_size:
            self.buffer.pop(0)  # Remove the oldest experience
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def size(self):
        return len(self.buffer)
    

import time

def train_model_with_replay(input_train1, target_train1, input_train2, target_train2, encoder, decoder, epochs, steps_per_epoch, batch_size, replay_buffer):
    
    
    for epoch in range(epochs):
        start = time.time()
        enc_hidden = encoder.initialize_hidden_state()
        total_loss = 0

        # Combine datasets
        combined_input = np.concatenate([input_train1, input_train2], axis=0)
        combined_target = np.concatenate([target_train1, target_train2], axis=0)

        # Define the train_dataset
        train_dataset = tf.data.Dataset.from_tensor_slices((combined_input, combined_target)).shuffle(10000).batch(batch_size, drop_remainder=True)

        # Train with mixed samples
        for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
            batch_loss = train_step(
                encoder, decoder, inp, targ, enc_hidden
            )
            total_loss += batch_loss

            # Add to replay buffer after training on first dataset
            if batch < len(input_train1) // batch_size:
                replay_buffer.add((inp, targ))

            # Mix with replay buffer samples during training on second dataset
            if batch >= len(input_train1) // batch_size:
                if replay_buffer.size() > 0:
                    replay_batch = replay_buffer.sample(batch_size)
                    replay_input, replay_target = zip(*replay_batch)
                    mixed_input = np.concatenate([inp, np.array(replay_input)], axis=0)
                    mixed_target = np.concatenate([targ, np.array(replay_target)], axis=0)

                    # Train with mixed data (from second dataset + replayed first dataset)
                    batch_loss = train_step(
                        encoder, decoder, mixed_input, mixed_target, enc_hidden
                    )
                    total_loss += batch_loss

            if batch % 100 == 0:
                print(f'Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy()}')

        print(f'Epoch {epoch + 1} Loss {total_loss / steps_per_epoch}')
        print(f'Time taken for 1 epoch {time.time() - start} sec\n')



In [61]:
gc.collect()

0

In [ ]:

# Initialize replay buffer
replay_buffer = ReplayBuffer(buffer_size=500)

EPOCHS = 15
train_model_with_replay(
    input_train_1,
    target_train_1,
    input_train_2,
    target_train_2,
    encoder_2,
    decoder_2,
    EPOCHS,
    steps_per_epoch_2,  # Example value
    batch_size,  # Example value
    replay_buffer=replay_buffer
)



Epoch 1 Batch 0 Loss 1.2946
Epoch 1 Loss 0.9923
Time taken for 1 epoch 444.6845715045929 sec

Epoch 2 Batch 0 Loss 1.0116
Epoch 2 Loss 0.8857
Time taken for 1 epoch 450.36882615089417 sec

Epoch 3 Batch 0 Loss 1.1224
Epoch 3 Loss 0.8326
Time taken for 1 epoch 449.26217555999756 sec

Epoch 4 Batch 0 Loss 0.9923
Epoch 4 Loss 0.7937
Time taken for 1 epoch 437.66591715812683 sec

Epoch 5 Batch 0 Loss 0.9658
Epoch 5 Loss 0.7527
Time taken for 1 epoch 431.86577677726746 sec

Epoch 6 Batch 0 Loss 0.7402
Epoch 6 Loss 0.7119
Time taken for 1 epoch 433.3895616531372 sec

Epoch 7 Batch 0 Loss 0.7080
Epoch 7 Loss 0.6676
Time taken for 1 epoch 433.9366111755371 sec

Epoch 8 Batch 0 Loss 0.7305
Epoch 8 Loss 0.6199
Time taken for 1 epoch 433.54721212387085 sec

Epoch 9 Batch 0 Loss 0.6322
Epoch 9 Loss 0.5637
Time taken for 1 epoch 432.2295410633087 sec

Epoch 10 Batch 0 Loss 0.6064
Epoch 10 Loss 0.5087
Time taken for 1 epoch 433.5595180988312 sec

Epoch 11 Batch 0 Loss 0.5857
Epoch 11 Loss 0.4498
Tim

In [63]:
def beam_evaluate_sentence(sentence,encoder,decoder,inp_lang_tokenizer,maxlen_input,targ_lang_tokenizer,beam_width=3):
    y = str(sentence)
    y = preprocess_eng(y)
    sentence= str(y)

    inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.strip().split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],maxlen=maxlen_input,padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''
    
    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)
    
    dec_h = enc_h
    dec_c = enc_c
    
    start_tokens = tf.fill([inference_batch_size], targ_lang_tokenizer.word_index['<start>'])
    end_token = targ_lang_tokenizer.word_index['<end>']
    
    enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
    decoder.attention_mechanism.setup_memory(enc_out)
    print("beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 194, 512]] :", enc_out.shape)
    
    # set decoder_inital_state which is an AttentionWrapperState considering beam_width
    hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
    decoder_initial_state = decoder.attention_decoder_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
    decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)
    
    # Instantiate BeamSearchDecoder
    decoder_instance = tfa.seq2seq.BeamSearchDecoder(decoder.attention_decoder_cell,beam_width=beam_width, output_layer=decoder.output_layer)
    decoder_embedding_matrix = decoder.decoder_embedding.variables[0]
    
    # The BeamSearchDecoder object's call() function takes care of everything.
    outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, 
                                                              start_tokens=start_tokens, end_token=end_token, 
                                                              initial_state=decoder_initial_state)
    
    final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
    beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))
    return final_outputs.numpy(), beam_scores.numpy()

In [64]:
def beam_translate2(sentence, targ_lang_tokenizer):
    result, beam_scores = beam_evaluate_sentence(sentence,encoder_2,decoder_2,inp_lang_tokenizer_2,maxlen_input_2,targ_lang_tokenizer)
    print(result.shape, beam_scores.shape)
    print()
    
    for beam, score in zip(result, beam_scores):
        print(beam.shape, score.shape)
        print()
        
    output = targ_lang_tokenizer.sequences_to_texts(beam)
    output = [a[:a.index('<end>')] for a in output]
    beam_score = [a.sum() for a in score]
    print('Input: %s' % (sentence))
    print()
    
    for i in range(len(output)):
        print('{} Predicted translation: {}  {}'.format(i+1, output[i], beam_score[i]))

In [65]:
print('dataset2:')
beam_translate2(u'Good omens appeared to Sita.',targ_lang_tokenizer_2)

beam_translate2(u'Valmiki then asks Narada a question.',targ_lang_tokenizer_2)


dataset2:
beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 194, 512]] : (3, 92, 512)
(1, 3, 8) (1, 3, 8)

(3, 8) (3, 8)

Input: Good omens appeared to Sita.

1 Predicted translation: ఇన్న నమ్ముకున్నాను !   -40.01940155029297
2 Predicted translation: ఇన్న సహాయంగా ఇంద్రుడు మాతలిని సారథిగా పంపాడు .   -42.65491485595703
3 Predicted translation: ఇన్న సమస్యలన్నీ చిటికెలో పరిష్కరమవాలంటే .   -46.907318115234375
beam_with * [batch_size, max_length_input, rnn_units] :  3 * [1, 194, 512]] : (3, 92, 512)
(1, 3, 12) (1, 3, 12)

(3, 12) (3, 12)

Input: Valmiki then asks Narada a question.

1 Predicted translation: ఆ అత్యుత్సాహంలో , భానుమతి ఏమైనా మితిమీరి పెట్టారు .   -89.1968002319336
2 Predicted translation: ఆ అత్యుత్సాహంలో , భానుమతి కూడా మితిమీరి , భ్రూమధ్యంబున అకస్మాత్తుగా .   -104.5013427734375
3 Predicted translation: ఆ అత్యుత్సాహంలో , భానుమతి ఏమైనా మితిమీరి , భ్రూమధ్యంబున అకస్మాత్తుగా ఉంది .   -110.79620361328125


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# Download NLTK data (if necessary)
nltk.download('punkt')


def evaluate_sen(sentence,encoder,decoder,inp_lang_tokenizer,maxlen_input,targ_lang_tokenizer):
    y = str(sentence)
    y = preprocess_eng(y)
    sentence= str(y)

    inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],maxlen=maxlen_input,padding='post')
    inputs = tf.convert_to_tensor(inputs)
    inference_batch_size = inputs.shape[0]
    result = ''
    
    enc_start_state = [tf.zeros((inference_batch_size, units)), tf.zeros((inference_batch_size,units))]
    enc_out, enc_h, enc_c = encoder(inputs, enc_start_state)
    dec_h = enc_h
    dec_c = enc_c
    
    start_tokens = tf.fill([inference_batch_size], targ_lang_tokenizer.word_index['<start>'])
    end_token = targ_lang_tokenizer.word_index['<end>']
    
    greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()
    
    # Instantiate BasicDecoder object
    decoder_instance = tfa.seq2seq.BasicDecoder(cell=decoder.attention_decoder_cell, sampler=greedy_sampler, 
                                                output_layer=decoder.output_layer)
    
    # Setup Memory in decoder stack
    decoder.attention_mechanism.setup_memory(enc_out)
    
    # set decoder_initial_state
    decoder_initial_state = decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)
    decoder_embedding_matrix = decoder.decoder_embedding.variables[0]
    
    outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, 
                                     end_token= end_token, initial_state=decoder_initial_state)
    return outputs.sample_id.numpy()


def translate2(sentence,targ_lang_tokenizer,reference):
  result = evaluate_sen(sentence,encoder_2,decoder_2,inp_lang_tokenizer_2,maxlen_input_2,targ_lang_tokenizer)
  print(result)
  result = targ_lang_tokenizer.sequences_to_texts(result)
  
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))
  
  # Calculate BLEU score
  bleu = calculate_bleu_score(reference, result)
  print(f"BLEU Score: {bleu:.4f}")
  

def calculate_bleu_score(reference, prediction):
    
    if isinstance(reference, list):
        reference = " ".join(reference)
    if isinstance(prediction, list):
        prediction = " ".join(prediction)
    prediction = prediction.replace('<end>', '').strip()
    # Tokenize the reference and prediction sentences
    reference_tokens = word_tokenize(reference)
    prediction_tokens = word_tokenize(prediction)

    # Wrap reference tokens in a list (BLEU requires list of lists for references)
    references = [reference_tokens]

    # Calculate the BLEU score using sentence_bleu
    bleu_score = sentence_bleu(references, prediction_tokens)
    return bleu_score






[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print('dataset2:')
# Example references (true translations, already in Telugu)
reference1 = 'పూజలు చేస్తుంటారు.'
translate2(u'They worship.',targ_lang_tokenizer_2,reference1)


dataset2:
[[1023 1694    4    3]]
Input: They worship.
Predicted translation: ['పూజలు చేస్తుంటారు . <end>']
BLEU Score: 0.1000


In [ ]:
reference2='మరి నీవద్ద అది ఉందా?'
translate2(u'And do you have it?',targ_lang_tokenizer_2,reference2)

[[ 14 152  98 465  10   3]]
Input: And do you have it?
Predicted translation: ['మీరు దానిని ఎందుకు కావాలి ? <end>']
BLEU Score: 0.1300


In [ ]:
print('dataset1:')
# Example references (true translations, already in Telugu)
reference1 = 'బాబు ముంబయిలో ఐఐటి చదువుతున్నాడు.'
reference2='సమాన పనికి సమాన వేతనం ఇవ్వాలన్నారు.'
reference3='దీంతో అధికారులు అలెర్ట్ అయ్యారు'
translate2(u'he is a student of iit at bombay.',targ_lang_tokenizer_2,reference1)
translate2(u'demand equal pay for equal work.',targ_lang_tokenizer_2,reference2)
translate2(u'the officials were shocked.',targ_lang_tokenizer_2, reference3)

dataset1:
[[5550 5551 5552 5553    4    3]]
Input: he is a student of iit at bombay.
Predicted translation: ['వ్యాపారస్తుడిని రాజుగారి ముందరు నిలపెట్టారు . <end>']
BLEU Score: 0.2300
[[  13 3349    5  662   19 1493 1576 3350    4    3]]
Input: demand equal pay for equal work.
Predicted translation: ['ఆ అత్యుత్సాహంలో , భానుమతి కూడా మితిమీరి పోలి సేవించింది . <end>']
BLEU Score: 0.1830
[[1668    4    3]]
Input: the officials were shocked.
Predicted translation: ['మంథాల్ . <end>']
BLEU Score: 0.0900
